In [0]:
x=1
y=4
print(x+y)

5


In [0]:
#Sum of the list
list = [1,2,3,4,5]
print(sum(list))

15


In [0]:
#Calculate average
print(sum(list)/len(list))

3.0


In [0]:
#tuple
my_tuple = (10,20,30)
print(my_tuple)

(10, 20, 30)


In [0]:
my_set = {1,2,3,4,5}
my_set.add(6)
print(my_set)
my_set.update([2,7,8]) #add multiple values
print(my_set)

{1, 2, 3, 4, 5, 6}
{1, 2, 3, 4, 5, 6, 7, 8}


In [0]:
#Dictionary
my_dict= {"name" : "Alice" , "age" : 25}
print(my_dict["name"]) # Output : Alice
my_dict["age"] = 26  # Updating value
print(my_dict.keys())  #dict_keys(['name','age'])
print(my_dict.values())

Alice
dict_keys(['name', 'age'])
dict_values(['Alice', 26])


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PythonDF").getOrCreate()

#Creating DataFrame from list of tuples
data =[(1,"Alice"),(2,"Bob"),(3,"Carol")]
df = spark.createDataFrame(data,["id","name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
+---+-----+



In [0]:
# Select specific columns
df.select("name").show()

# Filter rows
df.filter(df["id"] > 1).show()

# Count rows
print(df.count())

# Describe summary statistics (numeric columns)
df.describe().show()

# Add new column with literal value
from pyspark.sql.functions import lit
df = df.withColumn("country", lit("India"))
df.show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Carol|
+-----+

+---+-----+
| id| name|
+---+-----+
|  2|  Bob|
|  3|Carol|
+---+-----+

3
+-------+---+-----+
|summary| id| name|
+-------+---+-----+
|  count|  3|    3|
|   mean|2.0| null|
| stddev|1.0| null|
|    min|  1|Alice|
|    max|  3|Carol|
+-------+---+-----+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  1|Alice|  India|
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+



In [0]:
sales_data = [
    ("2024-01-01", "North", "Product A", 10, 200.0),
    ("2024-01-01", "South", "Product B", 5 , 300.0),
    ("2024-01-02", "North", "Product A", 20, 400.0),
    ("2024-01-02", "South", "Product B", 10, 600.0),
    ("2024-01-03", "East",  "Product C", 15, 375.0),
]
columns = ["date", "region", "product", "quantity", "revenue"]
sales_df = spark.createDataFrame(sales_data, columns)

sales_df.show()

+----------+------+---------+--------+-------+
|      date|region|  product|quantity|revenue|
+----------+------+---------+--------+-------+
|2024-01-01| North|Product A|      10|  200.0|
|2024-01-01| South|Product B|       5|  300.0|
|2024-01-02| North|Product A|      20|  400.0|
|2024-01-02| South|Product B|      10|  600.0|
|2024-01-03|  East|Product C|      15|  375.0|
+----------+------+---------+--------+-------+



In [0]:
# Total quantity and revenue by region
# • • groupBy0 is used to group data by a column.
# • • agg0 performs aggregation such as sumO and avg0.
# • • Aliases are used to rename columns for readability.
from pyspark.sql.functions import sum

# Calculate total revenue per product
total_revenue_df = sales_df.groupBy("product").agg(sum("revenue").alias("total_revenue"))
total_revenue_df.show()


#Total quantity and revenue by region
# Calculate total revenue per region
total_revenue_df = sales_df.groupBy("region").agg(sum("quantity").alias("Quantity"),sum("revenue").alias("Total_revenue"))
total_revenue_df.show()

+---------+-------------+
|  product|total_revenue|
+---------+-------------+
|Product A|        600.0|
|Product B|        900.0|
|Product C|        375.0|
+---------+-------------+

+------+--------+-------------+
|region|Quantity|Total_revenue|
+------+--------+-------------+
| North|      30|        600.0|
| South|      15|        900.0|
|  East|      15|        375.0|
+------+--------+-------------+



In [0]:
# Average Revenue per Product
from pyspark.sql.functions import avg

# Calculate average revenue per product
avg_revenue_df = sales_df.groupBy("product").agg((sum("revenue")/sum("quantity")).alias("avg_revenue"))
avg_revenue_df.show()

+---------+-----------+
|  product|avg_revenue|
+---------+-----------+
|Product A|       20.0|
|Product B|       60.0|
|Product C|       25.0|
+---------+-----------+



In [0]:
#Top-Perfom=rming Region (highest revenue)
from pyspark.sql.functions import sum

# Calculate total revenue per region and get the region with the highest revenue
top_region_df = (
    sales_df.groupBy("region")
    .agg(sum("revenue").alias("total_revenue"))
    .orderBy("total_revenue", ascending=False)
    .limit(1)
)

top_region_df.show()

+------+-------------+
|region|total_revenue|
+------+-------------+
| South|        900.0|
+------+-------------+



In [0]:
#Product name anf total revenue
from pyspark.sql.functions import sum
# sales.df.groupBy("product").agg(sum("revenue")).show()
display(sales_df.groupBy("product").agg(sum("revenue")))

product,sum(revenue)
Product A,600.0
Product B,900.0
Product C,375.0
